In [10]:
# some_file.py
import sys

sys.path.insert(0, './shared_code')



import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as db
import json
import pandas as pd
import importlib
from IPython.core.display import display, HTML

import os
import RHgenerate_states
import RHcomponents
import RHdisplay
import RHdistance_partition
import RHutilities as util

%matplotlib inline

In [2]:
con = db.connect('data/rush_hour.db')
cur = con.cursor()

#states.to_sql('states',con,if_exists="append")

In [7]:
importlib.reload(RHgenerate_states)
#importlib.reload(RHcomponents)

<module 'RHgenerate_states' from './shared_code\\RHgenerate_states.py'>

In [38]:

num_cars = 4
num_trucks = 2
combinatorial_class = 2**num_cars * 3**num_trucks
#RHgenerate_states.generate_states(num_cars,num_trucks,con)


In [5]:
len(states) , combinatorial_class

(18763346, 144)

In [ ]:
# prototype components algorithm using sqlite source for nodes

In [ ]:

comb_class = 144
solvable_count = 0
unsolvable_count = 0
cross_components_max_depth = 0

component_columns = ['comb_class','repr_board_int_s1','repr_board_int_s2',\
                     'repr_red_col', 'is_solvable','num_nodes',\
                     'density','max_solution_distance']

comb_class_columns = ['comb_class','num_nodes','num_components','num_solvable_components',\
                      'num_unsolvable_components','max_solution_depth']

select_node_sql = "select * from node where comb_class = 144 and included_in_component = 0 limit 1"
update_nodes_sql = ""




In [67]:
# Prime data for while condition
df = pd.read_sql_query(select_sql,con)
df

,id,comb_class,board_int_s1,board_int_s2,red_col,included_in_component
0,4,144,15437151837839400,2748779072896,1,0


In [ ]:
# create component
# seteup component data to store
# save component data
# upate node data to mark nodes as inluded in a component
# select next node not yet inlcuded in a component

total_nodes = 780 + 930 + 2400 + 1000 + 660
component_count = 5
while not df.empty:
    
    s1 = df.iloc[0]['board_int_s1']
    s2 = df.iloc[0]['board_int_s2']
    red_col = df.iloc[0]['red_col']
    board_int = util.combine_ints(s1,s2)
    board_int,red_col

    g = RHcomponents.components([(board_int,red_col)])[0]


    if not g.graph['solvable']:
        unsolvable_count +=1
        node_dict = g.node[0]
        max_soln_distance = None
    else:
        solvable_count +=1

        RHdistance_partition.distance_partition(g)

        max_soln_distance = max(g.graph['distance_partition'].keys())
        if max_soln_distance > cross_components_max_depth:
            cross_components_max_depth = max_soln_distance

        # select a node in outermost partition
        for node in g.graph['distance_partition'][max_soln_distance]:
            break
        node_dict = g.node[node]


    s1,s2 = util.split_int( node_dict['board_int'])    

    df_component = pd.DataFrame( [[\
                                     combinatorial_class
                                    ,s1
                                    ,s2
                                    ,node_dict['red_col']
                                    ,g.graph['solvable']
                                    ,len(g.nodes())
                                    ,nx.density(g)
                                    ,max_soln_distance                         
                                  ]],columns =  component_columns)

    df_component.to_sql('component',con,if_exists="append",index=False)        


    # Now update nodes
    cur = con.cursor()
    cur.executemany("update node set included_in_component = 1 where board_int_s1 = ? and board_int_s2 = ? and red_col = ?", \
                    [
        [  int(util.split_int(g.node[x]['board_int'])[0]),\
           int(util.split_int(g.node[x]['board_int'])[1]),\
           int(g.node[x]['red_col'])\
        ] for x in g.nodes() 
     ]
    )
    con.commit()


    df = pd.read_sql_query(select_sql,con)
    
    total_nodes = total_nodes + len(g.nodes())
    component_count += 1
    print ("%d | total nodes: %d"%(component_count,total_nodes) )
    


6 | total nodes: 6466
7 | total nodes: 7352
8 | total nodes: 7949
9 | total nodes: 8511
10 | total nodes: 8721
11 | total nodes: 8874
12 | total nodes: 9907
13 | total nodes: 10758
14 | total nodes: 10911
15 | total nodes: 11752
16 | total nodes: 11864
17 | total nodes: 12488
18 | total nodes: 13016
19 | total nodes: 13690
20 | total nodes: 14196
21 | total nodes: 14273
22 | total nodes: 15677
23 | total nodes: 16403
24 | total nodes: 17201
25 | total nodes: 17914
26 | total nodes: 18757
27 | total nodes: 21157
28 | total nodes: 21573
29 | total nodes: 22435
30 | total nodes: 23195
31 | total nodes: 24065
32 | total nodes: 24103
33 | total nodes: 24817
34 | total nodes: 25660
35 | total nodes: 26302
36 | total nodes: 26864
37 | total nodes: 27482
38 | total nodes: 27511
39 | total nodes: 28112
40 | total nodes: 28884
41 | total nodes: 29064
42 | total nodes: 29208
43 | total nodes: 29316
44 | total nodes: 29953
45 | total nodes: 30313
46 | total nodes: 30361
47 | total nodes: 30517
48 

331 | total nodes: 195314
332 | total nodes: 195890
333 | total nodes: 196466
334 | total nodes: 196528
335 | total nodes: 196605
336 | total nodes: 196729
337 | total nodes: 196838
338 | total nodes: 197498
339 | total nodes: 197506
340 | total nodes: 197535
341 | total nodes: 198230
342 | total nodes: 198258
343 | total nodes: 198990
344 | total nodes: 199679
345 | total nodes: 199682
346 | total nodes: 199685
347 | total nodes: 199742
348 | total nodes: 199832
349 | total nodes: 200548
350 | total nodes: 201122
351 | total nodes: 201764
352 | total nodes: 202383
353 | total nodes: 203304
354 | total nodes: 203367
355 | total nodes: 204354
356 | total nodes: 205455
357 | total nodes: 206025
358 | total nodes: 206806
359 | total nodes: 208054
360 | total nodes: 208204
361 | total nodes: 208357
362 | total nodes: 208519
363 | total nodes: 209905
364 | total nodes: 210050
365 | total nodes: 210674
366 | total nodes: 211173
367 | total nodes: 211297
368 | total nodes: 211937
369 | total 

648 | total nodes: 283172
649 | total nodes: 283256
650 | total nodes: 283313
651 | total nodes: 283481
652 | total nodes: 283490
653 | total nodes: 283510
654 | total nodes: 283542
655 | total nodes: 283587
656 | total nodes: 283607
657 | total nodes: 283967
658 | total nodes: 284021
659 | total nodes: 284036
660 | total nodes: 284069
661 | total nodes: 284267
662 | total nodes: 284277
663 | total nodes: 284421
664 | total nodes: 284455
665 | total nodes: 284489
666 | total nodes: 284583
667 | total nodes: 284617
668 | total nodes: 284773
669 | total nodes: 284819
670 | total nodes: 284881
671 | total nodes: 284935
672 | total nodes: 285479
673 | total nodes: 286082
674 | total nodes: 286118
675 | total nodes: 286170
676 | total nodes: 286918
677 | total nodes: 286954
678 | total nodes: 287587
679 | total nodes: 287681
680 | total nodes: 287686
681 | total nodes: 287764
682 | total nodes: 287840
683 | total nodes: 288092
684 | total nodes: 288808
685 | total nodes: 289064
686 | total 

964 | total nodes: 542657
965 | total nodes: 542837
966 | total nodes: 542981
967 | total nodes: 543089
968 | total nodes: 543293
969 | total nodes: 543489
970 | total nodes: 546220
971 | total nodes: 546322
972 | total nodes: 546760
973 | total nodes: 548577
974 | total nodes: 548667
975 | total nodes: 550195
976 | total nodes: 552119
977 | total nodes: 553705
978 | total nodes: 554721
979 | total nodes: 554947
980 | total nodes: 555097
981 | total nodes: 555849
982 | total nodes: 557834
983 | total nodes: 558414
984 | total nodes: 559262
985 | total nodes: 560349
986 | total nodes: 562700
987 | total nodes: 565513
988 | total nodes: 568769
989 | total nodes: 570350
990 | total nodes: 570512
991 | total nodes: 571384
992 | total nodes: 571454
993 | total nodes: 574498
994 | total nodes: 574968
995 | total nodes: 575476
996 | total nodes: 577719
997 | total nodes: 577773
998 | total nodes: 580194
999 | total nodes: 582779
1000 | total nodes: 582815
1001 | total nodes: 583293
1002 | tot

1271 | total nodes: 781219
1272 | total nodes: 781457
1273 | total nodes: 781485
1274 | total nodes: 781601
1275 | total nodes: 782042
1276 | total nodes: 782086
1277 | total nodes: 782262
1278 | total nodes: 782458
1279 | total nodes: 782703
1280 | total nodes: 782798
1281 | total nodes: 782994
1282 | total nodes: 783078
1283 | total nodes: 783565
1284 | total nodes: 784623
1285 | total nodes: 785665
1286 | total nodes: 786134
1287 | total nodes: 786164
1288 | total nodes: 786218
1289 | total nodes: 786526
1290 | total nodes: 786637
1291 | total nodes: 787096
1292 | total nodes: 787126
1293 | total nodes: 787164
1294 | total nodes: 787240
1295 | total nodes: 787297
1296 | total nodes: 787432
1297 | total nodes: 787477
1298 | total nodes: 789677
1299 | total nodes: 789746
1300 | total nodes: 789844
1301 | total nodes: 790754
1302 | total nodes: 791228
1303 | total nodes: 791714
1304 | total nodes: 792017
1305 | total nodes: 792641
1306 | total nodes: 792980
1307 | total nodes: 793530
1

...

10647 | total nodes: 7413809
10648 | total nodes: 7413862
10649 | total nodes: 7413878
10650 | total nodes: 7413954
10651 | total nodes: 7414083
10652 | total nodes: 7416040
10653 | total nodes: 7416069
10654 | total nodes: 7416537
10655 | total nodes: 7417005
10656 | total nodes: 7417240
10657 | total nodes: 7417250
10658 | total nodes: 7417427
10659 | total nodes: 7417718
10660 | total nodes: 7421194
10661 | total nodes: 7421248
10662 | total nodes: 7424114
10663 | total nodes: 7424429
10664 | total nodes: 7426264
10665 | total nodes: 7426285
10666 | total nodes: 7426853
10667 | total nodes: 7427527
10668 | total nodes: 7427695
10669 | total nodes: 7427903
10670 | total nodes: 7428206
10671 | total nodes: 7428526
10672 | total nodes: 7428650
10673 | total nodes: 7429123
10674 | total nodes: 7429151
10675 | total nodes: 7429731
10676 | total nodes: 7430097
10677 | total nodes: 7430167
10678 | total nodes: 7432671
10679 | total nodes: 7432759
10680 | total nodes: 7433787
10681 | total 

10930 | total nodes: 7570138
10931 | total nodes: 7570270
10932 | total nodes: 7570460
10933 | total nodes: 7570546
10934 | total nodes: 7572682
10935 | total nodes: 7573850
10936 | total nodes: 7574270
10937 | total nodes: 7574542
10938 | total nodes: 7574787
10939 | total nodes: 7574878
10940 | total nodes: 7577934
10941 | total nodes: 7578578
10942 | total nodes: 7578620
10943 | total nodes: 7578810
10944 | total nodes: 7578839
10945 | total nodes: 7579511
10946 | total nodes: 7579845
10947 | total nodes: 7580047
10948 | total nodes: 7580086
10949 | total nodes: 7580301
10950 | total nodes: 7580741
10951 | total nodes: 7581311
10952 | total nodes: 7581712
10953 | total nodes: 7581714
10954 | total nodes: 7581717
10955 | total nodes: 7581719
10956 | total nodes: 7581721
10957 | total nodes: 7581723
10958 | total nodes: 7581725
10959 | total nodes: 7583223
10960 | total nodes: 7583625
10961 | total nodes: 7583730
10962 | total nodes: 7583772
10963 | total nodes: 7583828
10964 | total 

11213 | total nodes: 7750236
11214 | total nodes: 7751206
11215 | total nodes: 7752082
11216 | total nodes: 7752095
11217 | total nodes: 7755621
11218 | total nodes: 7755706
11219 | total nodes: 7756067
11220 | total nodes: 7756252
11221 | total nodes: 7757247
11222 | total nodes: 7758297
11223 | total nodes: 7759347
11224 | total nodes: 7760347
11225 | total nodes: 7764464
11226 | total nodes: 7764531
11227 | total nodes: 7764607
11228 | total nodes: 7765410
11229 | total nodes: 7768983
11230 | total nodes: 7769795
11231 | total nodes: 7770079
11232 | total nodes: 7774305
11233 | total nodes: 7774735
11234 | total nodes: 7775313
11235 | total nodes: 7775480
11236 | total nodes: 7775583
11237 | total nodes: 7775617
11238 | total nodes: 7775712
11239 | total nodes: 7778175
11240 | total nodes: 7778713
11241 | total nodes: 7779074
11242 | total nodes: 7780358
11243 | total nodes: 7781152
11244 | total nodes: 7781776
11245 | total nodes: 7782400
11246 | total nodes: 7782519
11247 | total 

11496 | total nodes: 7906428
11497 | total nodes: 7906486
11498 | total nodes: 7906551
11499 | total nodes: 7906632
11500 | total nodes: 7906730
11501 | total nodes: 7906745
11502 | total nodes: 7907585
11503 | total nodes: 7907714
11504 | total nodes: 7907744
11505 | total nodes: 7908651
11506 | total nodes: 7908710
11507 | total nodes: 7909276
11508 | total nodes: 7909928
11509 | total nodes: 7910591
11510 | total nodes: 7911165
11511 | total nodes: 7911486
11512 | total nodes: 7912045
11513 | total nodes: 7912154
11514 | total nodes: 7916305
11515 | total nodes: 7917268
11516 | total nodes: 7917405
11517 | total nodes: 7917597
11518 | total nodes: 7919735
11519 | total nodes: 7920233
11520 | total nodes: 7920475
11521 | total nodes: 7920673
11522 | total nodes: 7921498
11523 | total nodes: 7921554
11524 | total nodes: 7922379
11525 | total nodes: 7922568
11526 | total nodes: 7922580
11527 | total nodes: 7922854
11528 | total nodes: 7922970
11529 | total nodes: 7923150
11530 | total 

11780 | total nodes: 7996651
11781 | total nodes: 7996769
11782 | total nodes: 7997160
11783 | total nodes: 7998097
11784 | total nodes: 7999069
11785 | total nodes: 7999856
11786 | total nodes: 7999892
11787 | total nodes: 8000084
11788 | total nodes: 8000326
11789 | total nodes: 8000522
11790 | total nodes: 8001902
11791 | total nodes: 8002136
11792 | total nodes: 8002703
11793 | total nodes: 8002868
11794 | total nodes: 8003434
11795 | total nodes: 8003647
11796 | total nodes: 8003930
11797 | total nodes: 8004337
11798 | total nodes: 8004579
11799 | total nodes: 8005267
11800 | total nodes: 8011752
11801 | total nodes: 8012050
11802 | total nodes: 8012897
11803 | total nodes: 8013712
11804 | total nodes: 8013805
11805 | total nodes: 8013885
11806 | total nodes: 8013925
11807 | total nodes: 8014174
11808 | total nodes: 8014317
11809 | total nodes: 8014440
11810 | total nodes: 8015112
11811 | total nodes: 8021517
11812 | total nodes: 8022342
11813 | total nodes: 8022522
11814 | total 

In [70]:
len(g.nodes()), len(g.nodes())  + 780

(930, 1710)

In [66]:
# now to update all 780 nodes with 
cur = con.cursor()
cur.executemany("update node set included_in_component = 1 where board_int_s1 = ? and board_int_s2 = ? and red_col = ?", \
                [
    [  int(util.split_int(g.node[x]['board_int'])[0]),\
       int(util.split_int(g.node[x]['board_int'])[1]),\
       int(g.node[x]['red_col'])\
    ] for x in g.nodes() 
 ]
)
con.commit()


In [ ]:
 [
    [  int(util.split_int(g.node[x]['board_int'])[0]),\
       int(util.split_int(g.node[x]['board_int'])[1]),\
       int(g.node[x]['red_col'])\
    ] for x in g.nodes() 
 ]


In [52]:
node_dict,g.graph['distance_partition'][11],s1,s2

({'board_int': 278140368219519116774982815580214,
  'inner_nbrs': {195, 197, 626, 675},
  'is_soln_state': False,
  'red_col': 1,
  'soln_distance': 11},
 {625},
 15439892043751464,
 133955638)

In [46]:

df_component

,comb_class,repr_board_int_s1,repr_board_int_s2,repr_red_col,is_solvable,num_nodes,density,max_solution_distance
0,144,15439892043751464,133955638,1,True,780,0.00891,11


In [35]:
g.graph

{'solvable': True}

In [22]:
#g = RHcomponents.components([(board_int,red_col)])[0]
if g.graph['solvable']:
    RHdistance_partition.distance_partition(g)

In [25]:
max(g.graph['distance_partition'].keys()) , nx.density(g)

(11, 0.008910174121984134)

In [ ]:
component

In [29]:
df_empty = pd.read_sql_query("select * from component where comb_class = 3",con)
df_empty.empty

True

In [ ]:

# prep for processing generated list of components
SAVE_BATCH_SIZE = 2000  # Number of Graphs to save in batch; Equals number of rows in DataFrame
db_components = []

comb_class_max_depth = 0

solvable_count = 0
unsolvable_count = 0
save_counter = 0

component_columns = ['comb_class','repr_board_int_s1','repr_board_int_s2',\
                     'repr_red_col', 'is_solvable','num_nodes',\
                     'density','max_solution_distance']

comb_class_columns = ['comb_class','num_nodes','num_components','num_solvable_components',\
                      'num_unsolvable_components','max_solution_depth']

for g in  RHcomponents.gen_components(states):
    
    save_counter +=1
    if save_counter >= SAVE_BATCH_SIZE:
        df = pd.DataFrame([x for x in db_components if x is not None],columns = component_columns)
        df.to_sql('component',con,if_exists="append")
        
        db_components = [None]*SAVE_BATCH_SIZE
        save_counter = 1
        
    if not g.graph['solvable']:
        unsolvable_count +=1
        node_dict = g.node[0]
        max_soln_distance = None
        
    else:
        
        solvable_count += 1
        
        RHdistance_partition.distance_partition(g)
        
        max_soln_distance = max(g.graph['distance_partition'].keys())
        if max_soln_distance > comb_class_max_depth:
            comb_class_max_depth = max_soln_distance
        for node in g.graph['distance_partition'][max_soln_distance]:
            break
        node_dict = g.node[node]
        
    
    s1,s2 = RHutilities.split_int( node_dict['board_int'])    
    db_components.append( [\
                             combinatorial_class
                            ,s1
                            ,s2
                            ,node_dict['red_col']
                            ,g.graph['solvable']
                            ,len(g.nodes())
                            ,nx.density(g)
                            ,max_soln_distance                         
                          ]
                        )
        
        
if  db_components:
    df = pd.DataFrame([x for x in db_components if x is not None],columns = component_columns)
    df.to_sql('component',con,if_exists="append",index=False)

    
#comb_class_columns = ['comb_class','num_nodes','num_components','num_solvable_components',\
#                      'num_unsolvable_components','max_solution_depth']

comb_class_data = [[combinatorial_class,num_nodes,unsolvable_count + solvable_count,\
                   solvable_count,unsolvable_count,  \
                   comb_class_max_depth]]

df_class = pd.DataFrame(comb_class_data,columns = comb_class_columns)
df_class.to_sql('comb_class',con,if_exists="append",index=False)

    

In [17]:
df

,comb_class,repr_board_int_s1,repr_board_int_s2,repr_red_col,is_solvable,num_nodes,density,max_solution_distance
0,4,221184,113246208,1,True,25,0.133333,4.0
1,4,224640,0,1,False,6,0.400000,NaN
2,4,223232,140737488355328,1,True,21,0.133333,5.0
3,4,906190848,0,1,True,25,0.133333,4.0
4,4,1269764,0,1,True,23,0.138340,5.0
5,4,221184,237494511599616,1,True,25,0.133333,4.0
6,4,221184,9007233614479360,1,True,23,0.138340,4.0
7,4,67330304,0,1,True,21,0.133333,5.0
8,4,221184,432,1,True,25,0.133333,4.0
9,4,1125904202031104,0,1,True,21,0.142857,4.0


In [18]:
df2 = pd.read_sql_query("select * from comb_class",con)
df2

,comb_class,num_nodes,num_components,num_solvable_components,num_unsolvable_components,max_solution_depth
0,4,267,13,12,1,5
1,36,116650,1361,1024,337,19


In [69]:
if g.graph['solvable']:
    RHdistance_partition.distance_partition(g)
    max_distance = max(g.graph['distance_partition'].keys())
    for node in g.graph['distance_partition'][max_distance]:
        break
    repr_node_dict = g.node[node]
else:
    repr_node_dict = g.node[0]
    max_distance = None

s1,s2 = RHutilities.split_int(repr_node_dict['board_int'])
red_col = repr_node_dict['red_col']
            
    
36, g.graph['solvable'],s1,s2,red_col, len(g.nodes()), nx.density(g),max_distance

(36, True, 67330304, 11259042018267128, 1, 204, 0.024244180430793006, 5)

In [11]:
len(comps),sum(len(x.nodes()) for x in comps)

(1361, 116650)

In [12]:
len(comps[0].nodes())

308

In [14]:
solvable_comps = [g for g in comps if g.graph['solvable']==True]
unsolvable_comps = [g for g in comps if g.graph['solvable'] == False]

In [15]:
df_solvable = pd.DataFrame([len(x.nodes()) for x in solvable_comps])
df_unsolvable = pd.DataFrame([len(x.nodes()) for x in unsolvable_comps])


## Here -
* Select representative for each component
* save json to file for each component (class_2_2_comp_39823484994949)
* draw graph with d3

* What I really want is real time back and forth to pull out components on the fly rather than build them all and save mass files to harddrive.



In [18]:
#df.describe()
#%matplotlib inline
df_solvable.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000250BD956EB8>]], dtype=object)

In [19]:
df_unsolvable.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000250BC07E9B0>]], dtype=object)

In [82]:
comp = comps[0]
min_int = min(comp.node[node]['board_int'] for node in comp.nodes())
min_int

632587360075802098027458330624

In [16]:
df_solvable.describe()

,0
count,1024.000000
mean,107.013672
std,100.871674
min,1.000000
25%,20.000000
50%,60.000000
75%,178.000000
max,400.000000


In [17]:
df_unsolvable.describe()

,0
count,337.000000
mean,20.973294
std,20.752708
min,1.000000
25%,6.000000
50%,15.000000
75%,27.000000
max,96.000000


In [83]:
#node = 0
comp_repr_node = [comp[node] for node in comp.nodes() if comp.node[node]['board_int'] == min_int]
comp_repr_node

[AtlasView({233: {}, 270: {}, 231: {}, 284: {}})]

In [113]:
len([g for g in comps if len(g.nodes())==1])

18

In [37]:
n = solvable_comps[40].node[0]

#HTML(RHdisplay.svg_from_state(solvable_comps[40].nodes(0)))

HTML(RHdisplay.svg_from_state(n['board_int'],n['red_col']))

In [39]:
df = pd.DataFrame( [  [len(g.nodes()),g.graph['solvable'] ] for g in comps])

In [55]:
g = solvable_comps[40]
g

In [58]:
RHdistance_partition.distance_partition(g)


{'board_int': 207885568651978971296788249575424,
 'inner_nbrs': {11, 49},
 'is_soln_state': False,
 'outer_nbrs': {9},
 'red_col': 2,
 'soln_distance': 4}

In [72]:
df_dist = pd.DataFrame( [g.node[n]['soln_distance'] for n in g.nodes() ], columns = ['distance'])

In [74]:
df_dist.groupby(['distance']).size()

distance
0    41
1    41
2    67
3    71
4    41
5    11
6     1
dtype: int64

In [76]:
HTML(RHdisplay.svg_from_state( g.node[0]['board_int'] , g.node[0]['red_col']))

In [77]:
len(solvable_comps) , len(unsolvable_comps)

(1024, 337)

In [79]:
len(solvable_comps) + len(unsolvable_comps)

1361

In [85]:
importlib.reload(RHdistance_partition)
for g in solvable_comps:
    RHdistance_partition.distance_partition(g)
    

In [142]:
for g in solvable_comps:
    g.graph['max_distance'] = max(g.graph['distance_partition'].keys())
    for node in g.graph['distance_partition'][g.graph['max_distance']]:
        break
    node_dict = g.node[node]
    split_board_int = RHutilities.split_int( node_dict['board_int'])
    g.graph['repr_board_int_s1'] = split_board_int[0]
    g.graph['repr_board_int_s2'] = split_board_int[1]
    g.graph['repr_red_col'] = node_dict['red_col']
    

In [162]:
df_max_dist = pd.DataFrame( \
            [ [g.graph['max_distance'],\
               g.graph['repr_board_int_s1'],\
               g.graph['repr_board_int_s2'],\
               g.graph['repr_red_col']\
               ,len(g.nodes())\
               ,g.size()\
               ,nx.density(g)\
              ]\
               for g in solvable_comps], columns=['distance','int_s1','int_s2','repr_red_col','num_nodes','num_edges','density'])

In [163]:
df_max_dist.sort_values(['distance'],ascending=False)

,distance,int_s1,int_s2,repr_red_col,num_nodes,num_edges,density
168,19,2748789558696,179633383279104,3,56,86,0.055844
769,17,2748789558696,175922545691136,3,80,149,0.047152
642,17,343598694837,25701168185664,4,116,260,0.038981
739,16,2748789776424,25701168185664,1,102,228,0.044263
830,15,2748789558696,670828544,3,84,172,0.049340
870,15,343598694837,21990330597696,4,144,344,0.033411
387,15,2748789558696,175922531535414,3,104,214,0.039955
572,15,21990316469568,670828544,2,86,161,0.044049
408,14,2748789776424,21990330597696,1,126,299,0.037968
506,14,175922531756544,68586307588,1,112,231,0.037162


In [164]:

board_int = RHutilities.combine_ints(21990316605814,11276634271252480)
red_col = 5
HTML(RHdisplay.svg_from_state(board_int,red_col))

In [166]:
df_max_dist.groupby(['distance']).agg(['count'])

,int_s1,int_s2,repr_red_col,num_nodes,num_edges,density
,count,count,count,count,count,count
distance,,,,,,
0,34,34,34,34,34,34
1,165,165,165,165,165,165
2,73,73,73,73,73,73
3,18,18,18,18,18,18
4,221,221,221,221,221,221
5,93,93,93,93,93,93
6,55,55,55,55,55,55
7,72,72,72,72,72,72


In [119]:
df_max_dist.sort_values(by=['distance'] , ascending = False)

,distance,int_s1,int_s2,repr_red_col
168,19,175922531535792,240243301154856,4
769,17,2748789558696,175922644781568,3
642,17,343598722053,25701168185664,2
739,16,28008,1924695198793728,2
830,15,140738035933224,2749850714112,1
870,15,27968,22341162893317,2
387,15,671312424,178670650027008,1
572,15,83886454,162728391737344,5
408,14,221504,24746353754152,1
506,14,175922531756544,343464214528,1


In [135]:
df_max_dist.loc[df_max_dist['distance'] == 19]
#HTML(RHdisplay.svg_from_state(n['board_int'],n['red_col']))

,distance,int_s1,int_s2,repr_red_col
168,19,175922531535792,240243301154856,4


In [137]:
board_int = RHutilities.combine_ints(2748789558696,240243301154856)
red_col = 4
HTML(RHdisplay.svg_from_state(board_int,red_col))

{0: {0}, 1: {1}}